# Information Extraction technique: Pure Regular Expression method
CARGILL notebook

In [ ]:
# install PyPDF2 package if it's not in your IDE
!pip install PyPDF2

Import necessary packages and Define the functions

In [ ]:
import re # regular expression(re) to formulate patterns.
import PyPDF2 # PyPDF2 to extract information from PDF files to plain text.
import pandas as pd # pandas to store extracted information to data frame data type

## Dictionary of Patterns for each attribute - Define patterns. Note that, if new files are added, check if the pattern is included or not.
PATTERNS_CARGILL = {
    "Product name": [
        r"Product name\s*:\s*(.+)",
        r"Trade name\s*:\s*(.*)"
    ],
    "Chemical": [
        r"Chemical name\s*:\s*([\w\s',-]+-C\d{2}-\d{2}-alkyl derivs\.)",
        r"Chemical name CAS-No. Concentration \(% w/w\)\s+([\w\s,-]+?(?:\n[\w\s,-]+)*)(?=\s+\d{2,7}-\d{2,3}-\d{1})",
        r"Chemical name\s*:\s*(.*)",
        r"Chemical Name\s*:\s*(.*\S)",
        r"\n([A-Za-z\s-]+) \d{2,5}-\d{2,3}-\d{1}"
    ],
    "CAS-No.": [
        r"(\d{2,6}-\d{2}-\d{1})",
        r"\b(\d{5}-\d{2}-\d{1})\b"
    ],
    "EC-No.": [
        r"(\n\d{3}-\d{3}-\d{1})"
    ],
    "Market Segment": [
        r"Use of the Sub-\nstance/Mixture\s+:\s+(.*)",
        r"Recommended use\s*:\s*(.+)",
        r"Use of the\s+Substance/Mixture\s*:\s*(.+)"
    ],
    "Supplier ID Description": [
        r"Company\s*:\s*(.+)",
        r"Company name of supplier\s+:\s+(.*)"
    ],
    "UN Code": [
        r"(UN\s\d{4})",
        r"(Not regulated as a dangerous good)"
    ],
    "Hazard": [
        r"GHS classification.*(?=Not a hazardous substance or mixture)",
        r"(Not a hazardous substance or mixture)"
    ]
}

def extract_text_from_pdf(file_path = str) -> str:
    # Define a function to extract everything from PDF to txt. file
    with open(file_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        all_text = [page.extract_text() for page in pdf_reader.pages]
    return "\n".join(all_text)

def search_patterns(patterns = str, text = str) -> str:
    # iterate through all the texts to find patterns listed in the PATTERNS_CARGILL dictionary
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1).replace('\n', ' ').strip()
    return "No information found"

def extract_info_from_text(pdf_text = str) -> pd:
    extracted_info = {}
    for attribute, patterns in PATTERNS_CARGILL.items():
        extracted_info[attribute] = search_patterns(patterns, pdf_text)

    # Set hazard value to 0 if it's found, otherwise 1
    extracted_info["Hazard"] = 0 if extracted_info["Hazard"] != "No information found" else 1
    return extracted_info

def extract_info_from_pdfs(file_paths = str) -> str:
    data = []
    for file_path in file_paths:
        pdf_text = extract_text_from_pdf(file_path)
        info = extract_info_from_text(pdf_text)
        info["File Path"] = file_path
        data.append(info)
    return data


Execution Code Block

In [ ]:
file_paths = [
    "CRODA-ANFOMUL 2000-SDS-EN.pdf",
    "CRODA-ANFOMUL 2500-SDS-EN.pdf",
    "CRODA-ANFOMUL S4M-SDS-EN.pdf",
    "CRODA-Atmer 100-SDS-EN.pdf",
    "CRODA-Atmer 1013-SDS-EN.pdf",
    "CRODA-Atmer 116-SDS-EN.pdf",
    "CRODA-Atmer 190-SDS-EN.pdf",
    "CRODA-Atmer 262-SDS-EN.pdf",
    "CRODA-Atmer 7306-SDS-EN.pdf",
    "CRODA-Crodamide 212-SDS-EN.pdf",
    "CRODA-Crodamide BR-SDS-EN.pdf",
    "CRODA-Crodamide EBO-SDS-EN.pdf",
    "CRODA-Crodamide EBS-SDS-EN.pdf",
    "CRODA-Crodamide ER DC08311-SDS-EN.pdf",
    "CRODA-Crodamide ER DC08312-SDS-EN.pdf",
    "CRODA-Crodamide O-SDS-EN.pdf",
    "CRODA-Crodamide VRX-SDS-EN.pdf",
    "CRODA-IONPHASE ESTAT2-PL-(FI)-SDS-EN.pdf",
    "CRODA-IONPHASE RSTAT2-PL-(FI)-SDS-EN.pdf",
    "CRODA-IONPHASE U1-PL-(FI)-SDS-EN.pdf",
    "CRODA-Incromold K-SDS-EN.pdf",
    "CRODA-Incroslip SL-SDS-EN.pdf",
    "CRODA-Ionphase Fstat-SDS-EN.pdf",
    "CRODA-Ionphase PE0108M-SDS-EN.pdf",
    "CRODA-Kemelix D309-SDS-EN.pdf",
    "CRODA-PERFAD FA 3340-LQ-(GD)-SDS-EN.pdf",
    "CRODA-PRIPOL 1022 VEG-LQ-(GD)-SDS-EN.pdf",
    "CRODA-Prifac 7990-SDS-EN.pdf",
    "CRODA-Priolube 3970-SDS-EN.pdf",
    "CRODA-Priplast 3192-SDS-EN.pdf",
    "CRODA-Pripol 1029-SDS-EN.pdf",
    "CRODA-Pripol 1040-SDS-EN.pdf",
    "CRODA-Prisorine 3505-SDS-EN.pdf",
    "CRODA-Prisorine 3515-SDS-EN.pdf",
    "CRODA-Pristerene 4963-SDS-EN.pdf",
    "EQUUS-KEMELIX D317-LQ-(MV)-SDS-EN.PDF",
    "CRODA-R05180B-TDS-EN.pdf"
] # Add all your PDF file paths here

# Extract information from all PDF files
data = extract_info_from_pdfs(file_paths)

# Convert the list of dictionaries into a DataFrame and display the DataFrame
df = pd.DataFrame(data)
df

## DataFrame to MS Excel

In [ ]:
df.to_excel("extracted_cargill.xlsx", index = False)

## DataFrame to SQL

In [ ]:
from sqlalchemy import create_engine
engine = create_engine("sqlite://", echo = False)

In [ ]:
df.to_sql(name = "product", con = engine)

In [ ]:
from sqlalchemy import text
with engine.connect() as conn:
    conn.execute(text("SELECT * FROM product")).fetchall()

In [ ]:
query = "SELECT* FROM product cargill"

In [ ]:
df_filtered = pd.read_sql_query(query, con = engine)
df_filtered